In [1]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit.Chem import MCS
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import rdFMCS
from rdkit.Chem import rdMolDescriptors
import timeit

import numpy as np
import pandas as pd
import seaborn as sns


#descriptor
from rdkit.Chem.Descriptors import ExactMolWt
from rdkit.Chem.Crippen import MolLogP
from rdkit.Chem.rdMolDescriptors import CalcNumHBD
from rdkit.Chem.rdMolDescriptors import CalcNumHBA
from rdkit.Chem.rdMolDescriptors import CalcTPSA

import matplotlib.pyplot as plt

/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The rdkit.Chem.MCS module is deprecated; please use rdkit.Chem.rdFMCS instead.
  after removing the cwd from sys.path.


In [2]:
data=pd.read_csv('pubchem_filtered_20210824.csv')

In [3]:
#data=pd.read_csv('train.txt',header=None)

In [4]:
data

,CID,smiles,MW,LogP,HBA,HBD,TPSA,HBA_HBD,rotational bond,form_charge,length
0,140,CC(CCCC(C)C1CCC2C1(C(CC3C2C(CC4C3(CCC(C4)O)C)O...,436.355260,4.38260,4,4,80.92,8,6,0,53
1,159,CCCCCC(C=CC1C(CC2C1CC(=CCCCC(=O)O)O2)O)O,352.224974,3.40850,4,3,86.99,7,10,0,40
2,246,CC12CCC(CC1CCC3C2CCC4(C3CCC4=O)C)OS(=O)(=O)O,370.181395,3.78620,4,1,80.67,5,2,0,44
3,302,CC(CCC(=O)O)C1CCC2C1(C(CC3C2C(=O)CC4C3(CCC(C4)...,406.271924,3.65690,4,3,94.83,7,4,0,53
4,303,CC(CCC(=O)O)C1CCC2C1(C(CC3C2C(CC4C3(CCC(C4)O)C...,408.287574,3.44870,4,4,97.99,8,4,0,52
5,699,CC12CCC3C(C1CCC2=O)CCC4=C3C=CC(=C4)OS(=O)(=O)O,350.118795,3.29340,4,1,80.67,5,2,0,46
6,734,CC(=CCCC(=CCOP(=O)(O)OP(=O)(O)O)C)C,314.068426,2.90540,4,3,113.29,7,8,0,35
7,1020,CC(=CCCC=CCOP(=O)(O)OP(=O)(O)O)C,300.052776,2.51530,4,3,113.29,7,8,0,32
8,1199,CC1=CC=C(C=C1)S(=O)(=O)N(CC(=O)O)C2=CC(=CC=C2)Cl,339.033207,2.92832,3,1,74.68,4,5,0,48
9,1239,CC1(CCC(CC1)C(C)(C)NC(=O)CBr)NCC(COC2=CC=CC3=C...,479.178354,3.73580,4,4,86.38,8,9,0,54


In [5]:
rand_sample=data.sample(n=100000)

In [6]:
rand_sample

,CID,smiles,MW,LogP,HBA,HBD,TPSA,HBA_HBD,rotational bond,form_charge,length
12195429,141733997,C1CNCC1NC(=O)[C@H]2C3C4=C(C=CN=C4S2)N(C(=O)N3)...,507.113188,4.38210,6,3,95.59,9,5,0,75
4258462,46981123,COC1=C(C=CC(=C1)CN2CCCC(C2)CC3=NC4=CC=CC=C4N3)...,408.216141,2.89030,5,2,93.47,7,8,0,54
12405451,143753685,CC=N.C#C.C1=CC(=CC=C1C2=C(N3C(=N2)C=CC(=N3)Cl)...,387.101751,4.62327,5,2,80.06,7,3,0,56
4197028,46648407,CC1=CC(=C(C(=C1)C)NC(=O)C(C)OC(=O)CCC2=C(N(N=C...,413.267842,4.58420,5,1,73.22,6,8,0,59
8401337,102480398,C1=CC(=CC=C1NC2=C(C3=C(C=CC(=C3)Cl)NC2=O)C(=O)...,348.006848,4.27670,3,3,82.19,6,3,0,50
5220339,56429703,CC1=CC=C(C=C1)OCCC(=O)OCC(=O)NC2=C(C=NN2C3=CC=...,404.148455,3.00320,7,1,106.24,8,8,0,55
12542581,145068478,CC1=C2C(=NC=NN2C(=C1)C3=CC(=C(C=C3)SC)N4CCN(C(...,438.183795,2.98252,7,1,96.83,8,3,0,64
5440655,57951666,CC(C)([C@H]1C(=O)N[C@@H](C(=O)NCCCC2=CC=CC=C2O...,592.362471,2.99170,6,4,120.00,10,4,0,87
1532615,11725442,CC(C)(C)C1=CC(=NN1)NC(=O)C2=C(N=CC=C2)NCC3=CC=...,350.185509,3.36160,5,3,95.59,8,5,0,51
4939340,55689053,CC(C(=O)NC1=CC(=CC=C1)OCC2=CC(=CC=C2)C#N)SCC(=...,463.136591,4.97528,5,2,91.22,7,9,0,63


In [7]:
data=rand_sample

In [8]:
#filter

data=data[~data.smiles.str.contains("Pb")]
data=data[~data.smiles.str.contains("Cu")]
data=data[~data.smiles.str.contains("Se")]
data=data[~data.smiles.str.contains("Ac")]
data=data[~data.smiles.str.contains("Tc")]
data=data[~data.smiles.str.contains("As")]
data=data[~data.smiles.str.contains("Ni")]

data=data[~data.smiles.str.contains("Eu")]
data=data[~data.smiles.str.contains("Ru")]
data=data[~data.smiles.str.contains("Ti")]
data=data[~data.smiles.str.contains("Li")]
data=data[~data.smiles.str.contains("Ta")]
data=data[~data.smiles.str.contains("Fe")]
data=data[~data.smiles.str.contains("Pt")]
data=data[~data.smiles.str.contains("C-")]
data=data[~data.smiles.str.contains("U")]

data=data[~data.smiles.str.contains("Hg")]
data=data[~data.smiles.str.contains("Zn")]
data=data[~data.smiles.str.contains("Pd")]
data=data[~data.smiles.str.contains("S-")]
data=data[~data.smiles.str.contains("Ga")]
data=data[~data.smiles.str.contains("Ge")]

data=data[~data.smiles.str.contains("V")]
data=data[~data.smiles.str.contains("W")]
data=data[~data.smiles.str.contains("Y")]
data=data[~data.smiles.str.contains("Tl")]
data=data[~data.smiles.str.contains("Rh")]
data=data[~data.smiles.str.contains("Re")]

data=data[~data.smiles.str.contains("Zr")]
data=data[~data.smiles.str.contains("Cd")]
data=data[~data.smiles.str.contains("Rb")]
data=data[~data.smiles.str.contains("Sb")]
data=data[~data.smiles.str.contains("Mn")]
data=data[~data.smiles.str.contains("Mo")]
data=data[~data.smiles.str.contains("Gd")]
data=data[~data.smiles.str.contains("Ag")]
data=data[~data.smiles.str.contains("Tb")]
data=data[~data.smiles.str.contains("Rf")]
data=data[~data.smiles.str.contains("Ba")]
data=data[~data.smiles.str.contains("Sm")]
data=data[~data.smiles.str.contains("Hf")]
data=data[~data.smiles.str.contains("Lu")]
data=data[~data.smiles.str.contains("La")]

data=data[~data.smiles.str.contains("Ce")]


In [9]:
data

,CID,smiles,MW,LogP,HBA,HBD,TPSA,HBA_HBD,rotational bond,form_charge,length
12195429,141733997,C1CNCC1NC(=O)[C@H]2C3C4=C(C=CN=C4S2)N(C(=O)N3)...,507.113188,4.38210,6,3,95.59,9,5,0,75
4258462,46981123,COC1=C(C=CC(=C1)CN2CCCC(C2)CC3=NC4=CC=CC=C4N3)...,408.216141,2.89030,5,2,93.47,7,8,0,54
12405451,143753685,CC=N.C#C.C1=CC(=CC=C1C2=C(N3C(=N2)C=CC(=N3)Cl)...,387.101751,4.62327,5,2,80.06,7,3,0,56
4197028,46648407,CC1=CC(=C(C(=C1)C)NC(=O)C(C)OC(=O)CCC2=C(N(N=C...,413.267842,4.58420,5,1,73.22,6,8,0,59
8401337,102480398,C1=CC(=CC=C1NC2=C(C3=C(C=CC(=C3)Cl)NC2=O)C(=O)...,348.006848,4.27670,3,3,82.19,6,3,0,50
5220339,56429703,CC1=CC=C(C=C1)OCCC(=O)OCC(=O)NC2=C(C=NN2C3=CC=...,404.148455,3.00320,7,1,106.24,8,8,0,55
12542581,145068478,CC1=C2C(=NC=NN2C(=C1)C3=CC(=C(C=C3)SC)N4CCN(C(...,438.183795,2.98252,7,1,96.83,8,3,0,64
5440655,57951666,CC(C)([C@H]1C(=O)N[C@@H](C(=O)NCCCC2=CC=CC=C2O...,592.362471,2.99170,6,4,120.00,10,4,0,87
1532615,11725442,CC(C)(C)C1=CC(=NN1)NC(=O)C2=C(N=CC=C2)NCC3=CC=...,350.185509,3.36160,5,3,95.59,8,5,0,51
4939340,55689053,CC(C(=O)NC1=CC(=CC=C1)OCC2=CC(=CC=C2)C#N)SCC(=...,463.136591,4.97528,5,2,91.22,7,9,0,63


In [10]:
smiles=data['smiles']

In [11]:
len(smiles)

99816

In [12]:
smiles.to_csv('random_filtered_pubchem20210824.txt',index=False)

/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [13]:
#rand_sample.to_csv('random_20210524.csv',index=False)